使用least_squares库中LM算法

## 步骤1：计算T0_3变换矩阵

In [13]:
import numpy as np
from scipy.optimize import least_squares

def compute_T0_3(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, theta4, theta5, theta6, dh_params):
    """
    计算中间变换矩阵T0_3
    
    参数:
    x, y, z: TCP位置坐标
    nx, ny, nz, ox, oy, oz, ax, ay, az: TCP姿态矩阵元素
    theta4, theta5, theta6: 已知的关节4,5,6角度(弧度)
    dh_params: 机器人的DH参数字典
    
    返回:
    T0_3: 从基座标系到关节3坐标系的变换矩阵
    """
    # 1. 构建总体变换矩阵T
    T = np.array([
        [nx, ox, ax, x],
        [ny, oy, ay, y],
        [nz, oz, az, z],
        [0, 0, 0, 1]
    ])
    
    # 2. 计算T3_6 (关节3到关节6的变换)
    # 提取DH参数
    a3 = dh_params.get('a3', 0)
    a4 = dh_params.get('a4', 0)
    a5 = dh_params.get('a5', 0)
    d4 = dh_params.get('d4', 0)
    d5 = dh_params.get('d5', 0)
    d6 = dh_params.get('d6', 0)
    alpha3 = dh_params.get('alpha3', 0)
    alpha4 = dh_params.get('alpha4', 0)
    alpha5 = dh_params.get('alpha5', 0)
    
    # 计算T3_4, T4_5, T5_6
    T3_4 = dh_transform(a3, alpha3, d4, theta4)
    T4_5 = dh_transform(a4, alpha4, d5, theta5)
    T5_6 = dh_transform(a5, alpha5, d6, theta6+180*np.pi/180)  # 注意theta6需要加上180度
    
    # 计算T3_6
    T3_6 = np.dot(np.dot(T3_4, T4_5), T5_6)
    
    # 3. 计算T0_3 = T * (T3_6)^-1
    T3_6_inv = np.linalg.inv(T3_6)
    T0_3 = np.dot(T, T3_6_inv)
    print("T0_3:")
    print(T0_3)
    return T0_3

def dh_transform(a, alpha, d, theta):
    """
    计算DH变换矩阵
    
    参数:
    a: 连杆长度
    alpha: 连杆扭角(弧度)
    d: 连杆偏距
    theta: 关节角度(弧度)
    
    返回:
    T: 4x4的DH变换矩阵
    """
    # 构建DH变换矩阵
    ct = np.cos(theta)
    st = np.sin(theta)
    ca = np.cos(alpha)
    sa = np.sin(alpha)
    
    T = np.array([
        [ct, -st, 0, a],
        [st*ca, ct*ca, -sa, -d*sa],
        [st*sa, ct*sa, ca, d*ca],
        [0, 0, 0, 1]
    ])
    
    return T

## 步骤2：使用LM算法求解关节角度
现在我们有了T0_3矩阵，需要找到关节角度theta1, theta2, theta3，使得这些角度产生的变换矩阵尽可能接近计算出的T0_3。这是一个非线性最小二乘问题，非常适合使用Levenberg-Marquardt算法求解。

自定义机器人雅可比矩阵计算方法

In [14]:
def custom_jacobian(angles, dh_params):
        """
        计算残差函数对关节角度的雅可比矩阵
        
        参数:
            angles: 当前关节角度 [theta1, theta2, theta3]
            
        返回:
            6×3的雅可比矩阵，每列代表对应关节的影响
        """
        theta1, theta2, theta3 = angles
          # 提取DH参数
        a1 = dh_params.get('a1', 0)
        a2 = dh_params.get('a2', 0)
        d1 = dh_params.get('d1', 0)
        d2 = dh_params.get('d2', 0)
        d3 = dh_params.get('d3', 0)
        alpha1 = dh_params.get('alpha1', 0)
        alpha2 = dh_params.get('alpha2', 0)
        # 计算各个变换矩阵
        T0_1 = dh_transform(0, 0, d1, theta1)
        T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)
        T2_3 = dh_transform(a2, alpha2, d3, theta3)
        
        # 计算T0_2和T0_3
        T0_2 = np.dot(T0_1, T1_2)
        T0_3_current = np.dot(T0_2, T2_3)
        
        # 计算各关节坐标系的z轴方向（旋转轴）
        z0 = np.array([0, 0, 1])  # 基坐标系的z轴
        z1 = T0_1[:3, 2]          # 关节1坐标系的z轴
        z2 = T0_2[:3, 2]          # 关节2坐标系的z轴
        
        # 计算各关节的位置
        o0 = np.array([0, 0, 0])
        o1 = T0_1[:3, 3]
        o2 = T0_2[:3, 3]
        o3 = T0_3_current[:3, 3]  # 末端位置
        
        # 计算位置雅可比矩阵 (3×3)
        J_v = np.zeros((3, 3))
        
        # 关节1对末端位置的影响
        J_v[:, 0] = np.cross(z0, o3 - o0)
        
        # 关节2对末端位置的影响
        J_v[:, 1] = np.cross(z1, o3 - o1)
        
        # 关节3对末端位置的影响
        J_v[:, 2] = np.cross(z2, o3 - o2)
        
        # 计算旋转雅可比矩阵 (3×3)
        J_w = np.zeros((3, 3))
        J_w[:, 0] = z0  # 关节1的旋转轴
        J_w[:, 1] = z1  # 关节2的旋转轴
        J_w[:, 2] = z2  # 关节3的旋转轴
        
        # 应用权重
        position_weight = 1.0
        rotation_weight = 0.1
        
        # 合并为完整的雅可比矩阵 (6×3)
        J = np.vstack([J_v, J_w])
        
        return J



In [15]:
def solve_joint_angles_lm_improved(T0_3, dh_params, initial_guess=None):
    """改进的LM算法求解关节角度"""
    # 提取DH参数
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    
    def improved_residuals(angles):
        theta1, theta2, theta3 = angles
        # 计算各关节变换矩阵
        T0_1 = dh_transform(0,0, d1, theta1)
        T1_2 = dh_transform(a1, alpha1, d2, theta2-90*np.pi/180)
        T2_3 = dh_transform(a2, alpha2, d3, theta3)
        
        # 计算预测的T0_3
        T0_3_pred = np.dot(np.dot(T0_1, T1_2), T2_3)
        
        # 计算位置误差
        position_error = T0_3_pred[:3, 3] - T0_3[:3, 3]
        
        # 计算旋转误差 (使用对数映射)
        R_pred = T0_3_pred[:3, :3]
        R_target = T0_3[:3, :3]
        R_error = np.dot(R_pred, R_target.T)
        # 将旋转矩阵转换为轴角表示
        theta = np.arccos((np.trace(R_error) - 1) / 2)
        if theta < 1e-10:
            rotation_error = np.zeros(3)
        else:
            # 计算旋转误差 k*sin(θ) = (r32-r23, r13-r31, r21-r12)/2
            rotation_error = theta * np.array([
                # 计算旋转误差的x分量
                R_error[2, 1] - R_error[1, 2],
                # 计算旋转误差的y分量
                R_error[0, 2] - R_error[2, 0],
                # 计算旋转误差的z分量
                R_error[1, 0] - R_error[0, 1]
            ]) / (2 * np.sin(theta))

        # 设置位置和旋转的权重
        position_weight = 1.0
        rotation_weight = 1.0
        
        # 组合所有误差项
        residuals = np.concatenate([
            position_weight * position_error,
            rotation_weight * rotation_error
        ])
    
        return residuals 
    
    # 设置初始猜测值
    if initial_guess is None:
        initial_guess = [0.0, 0.0, 0.0]
    
    # 使用改进的LM算法参数
   # 调整 LM 求解器参数
    result = least_squares(
        improved_residuals, 
        initial_guess,
        method='lm',
        #jac=custom_jacobian,  # 计算雅可比矩阵
        ftol=1e-14,      # 提高精度
        xtol=1e-14,
        gtol=1e-14,
        max_nfev=10000,   # 增加最大迭代次数
        f_scale=0.05,     # 调整损失函数尺度
        tr_solver='exact'# 使用精确求解器
    )
    
    
    # 获取计算结果
    theta1, theta2, theta3 = result.x
    
    # 规范化角度到[-pi, pi]范围域
    theta1 = ((theta1 + np.pi) % (2*np.pi)) - np.pi
    theta2 = ((theta2 + np.pi) % (2*np.pi)) - np.pi
    theta3 = ((theta3 + np.pi) % (2*np.pi)) - np.pi
    
    return theta1, theta2, theta3

## 步骤3：多初始值策略以避免局部最小值

In [ ]:
def solve_joint_angles_multi_init(T0_3, dh_params):
    """使用多个初始猜测值提高求解成功率"""
    # 设置多个初始猜测值
    initial_guesses = [
        [0.0, 0.0, 0.0],
        [np.pi/4, np.pi/4, np.pi/4],
        [-np.pi/4, np.pi/4, np.pi/4],
        [np.pi/4, -np.pi/4, np.pi/4],
        [np.pi/4, np.pi/4, -np.pi/4],
        [np.pi/2, 0.0, 0.0],
        [0.0, np.pi/2, 0.0],
        [0.0, 0.0, np.pi/2],
        [np.pi/3, np.pi/3, np.pi/3],
        [-np.pi/3, -np.pi/3, -np.pi/3],
        [np.pi/6, np.pi/6, np.pi/6],
        [-np.pi/6, -np.pi/6, -np.pi/6]
    ]
    
    best_solution = None
    min_error = float('inf')
    
    for guess in initial_guesses:
        try:
            # 尝试求解
            theta1, theta2, theta3 = solve_joint_angles_lm_improved(T0_3, dh_params, guess)
            
            # 验证解的质量
            error = compute_solution_error(theta1, theta2, theta3, T0_3, dh_params)
            
            if error < min_error:
                min_error = error
                best_solution = (theta1, theta2, theta3)
        except:
            continue
    
    if best_solution is not None:
        return best_solution, min_error
    else:
        raise ValueError("无法找到有效解")

def compute_solution_error(theta1, theta2, theta3, target_T0_3, dh_params):
    """计算解的误差"""
    # 提取DH参数
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    
    # 计算各关节变换矩阵

    T0_1 = dh_transform(0,0, d1, theta1)
    T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)  # 第二个关节有90度的偏移
    T2_3 = dh_transform(a2, alpha2, d3, theta3)  # 
    
    # 计算预测的T0_3
    T0_3_pred = np.dot(np.dot(T0_1, T1_2), T2_3)
    
    # 计算误差
    error_matrix = T0_3_pred - target_T0_3
    
    # 计算Frobenius范数作为总体误差
    return np.linalg.norm(error_matrix[:3, :])

## 完整的求解

In [17]:
def solve_inverse_kinematics_lm(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                              theta4, theta5, theta6, dh_params):
    """
    使用LM方法求解机器人反向运动学
    
    参数:
    x, y, z: TCP位置坐标
    nx, ny, nz, ox, oy, oz, ax, ay, az: TCP姿态矩阵元素
    theta4, theta5, theta6: 已知的关节4,5,6角度(弧度)
    dh_params: 机器人的DH参数字典
    
    返回:
    theta1, theta2, theta3: 计算得到的关节1、2、3的角度(弧度)
    error: 解的误差
    """
    # 1. 计算T0_3
    T0_3 = compute_T0_3(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                        theta4, theta5, theta6, dh_params)
    
    # 2. 使用多初始值策略求解关节角度
    (theta1, theta2, theta3), error = solve_joint_angles_multi_init(T0_3, dh_params)
    
    # 3. 打印结果和误差
    print(f"求解结果:")
    print(f"关节1角度: {np.degrees(theta1):.2f}°")
    print(f"关节2角度: {np.degrees(theta2):.2f}°")
    print(f"关节3角度: {np.degrees(theta3):.2f}°")
    print(f"误差: {error:.6f}")
    
    return theta1, theta2, theta3, error

In [ ]:

from scipy.spatial.transform import Rotation 

# TCP位置坐标和姿态  
x, y, z = 915.249, 608.436, 233.411  # 位置(mm)  
quat = [0.3604,0.8224,0.3919,-0.2006] #[x y z w]  # 四元数
# 已知的关节4,5,6角度(弧度)  
theta4 = np.radians(90)  
theta5 = np.radians(45)  
theta6 = np.radians(0)  

# 创建旋转对象
rot = Rotation.from_quat(quat)

# 获取旋转矩阵
rotation_matrix = rot.as_matrix() 

# 提取姿态矩阵的各列向量  
nx,ny,nz = rotation_matrix[:,0]  # x轴方向向量
ox,oy,oz = rotation_matrix[:,1]  # y轴方向向量  
ax,ay,az = rotation_matrix[:,2] 

# 已知的关节4,5,6角度(弧度)  
theta4 = np.radians(90)  
theta5 = np.radians(45)  
theta6 = np.radians(0)  

# 设置机器人DH参数(工业机器人 NB12s-1214-5A)  
dh_params = {  
    'a0': 0,      # 连杆1长度(mm)
    'a1': 100,    # 连杆2长度(mm) 
    'a2': 680,    # 连杆3长度(mm)
    'a3': 50,    # 连杆4长度(mm)
    'a4': 0,      # 连杆5长度(mm)
    'a5': 0,      # 连杆6长度(mm)
    'd1': 500,    # 关节1偏移(mm)
    'd2': 0,      # 关节2偏移(mm)
    'd3': 0,      # 关节3偏移(mm
    'd4': 660,    # 关节4偏移(mm)
    'd5': 0,      # 关节5偏移(mm)
    'd6': 98,    # 关节6偏移(mm)，末端执行器偏移
    'alpha0': 0,           # 连杆0扭角(弧度)
    'alpha1': -np.pi/2,   # 连杆1扭角(弧度)
    'alpha2': 0,          # 连杆2扭角(弧度)
    'alpha3': -np.pi/2,   # 连杆3扭角(弧度)
    'alpha4': np.pi/2,    # 连杆4扭角(弧度)
    'alpha5': -np.pi/2,   # 连杆5扭角(弧度)
}

try:
    # 使用LM方法求解关节角度
    theta1, theta2, theta3, error = solve_inverse_kinematics_lm(
        x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
        theta4, theta5, theta6, dh_params
    )
    
    # 验证求解结果
    print("\n验证求解结果:")
    
    # 计算正向运动学
    # 计算变换矩阵T0_1, T1_2, T2_3, T3_4, T4_5, T5_6
    T0_1 = dh_transform(0, 0, dh_params['d1'], theta1)
    T1_2 = dh_transform(dh_params['a1'], dh_params['alpha1'], dh_params['d2'], theta2-np.pi/2)  # 第二个关节有90度的偏移
    T2_3 = dh_transform(dh_params['a2'], dh_params['alpha2'], dh_params['d3'], theta3)
    T3_4 = dh_transform(dh_params['a3'], dh_params['alpha3'], dh_params['d4'], theta4)
    T4_5 = dh_transform(0, dh_params['alpha4'], dh_params['d5'], theta5)
    T5_6 = dh_transform(0, dh_params['alpha5'], dh_params['d6'], theta6+np.pi)  # 注意theta6需要加上180度

    #计算T03
    T0_3 = np.dot(np.dot(T0_1, T1_2), T2_3)
    print("计算T0_3:")
    print(T0_3)
    # 计算总体变换矩阵
    T = np.dot(T0_1, np.dot(T1_2, np.dot(T2_3, np.dot(T3_4, np.dot(T4_5, T5_6)))))
    
    # 提取TCP位置和姿态
    x_verify, y_verify, z_verify = T[0:3, 3]
    
    # 计算位置误差
    position_error = np.sqrt((x-x_verify)**2 + (y-y_verify)**2 + (z-z_verify)**2)
    
    print(f"原始TCP位置: [{x}, {y}, {z}]")
    print(f"验证TCP位置: [{x_verify:.3f}, {y_verify:.3f}, {z_verify:.3f}]")
    print(f"位置误差: {position_error:.2f} mm")
    
    # 比较姿态矩阵
    rotation_error = np.linalg.norm(rotation_matrix - T[0:3, 0:3])
    print(f"姿态误差: {rotation_error:.6f}")
    
    # 打印所有关节角度
    print("\n完整关节角度:")
    print(f"关节1: {np.degrees(theta1):.2f}°")
    print(f"关节2: {np.degrees(theta2):.2f}°")
    print(f"关节3: {np.degrees(theta3):.2f}°")
    print(f"关节4: {np.degrees(theta4):.2f}°")
    print(f"关节5: {np.degrees(theta5):.2f}°")
    print(f"关节6: {np.degrees(theta6):.2f}°")
    
except Exception as e:
    print(f"求解过程出错: {e}")
    
    # 尝试使用不同的初始值或其他求解方法
    print("尝试使用备用求解方法...")
    # 这里可以添加备用求解方法，如几何法、解析法等

T0_3:
[[ 7.49974284e-01  4.32960965e-01 -5.00083369e-01  5.96647394e+02]
 [ 4.33084145e-01  2.50042996e-01  8.65977265e-01  3.44417157e+02]
 [ 4.99976696e-01 -8.66038858e-01  1.72877946e-05  8.40010044e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
求解结果:
关节1角度: 30.00°
关节2角度: 60.00°
关节3角度: 0.00°
误差: 0.025034

验证求解结果:
计算T0_3:
[[ 7.50042887e-01  4.33010541e-01 -4.99937535e-01  5.96628619e+02]
 [ 4.32965335e-01  2.49957112e-01  8.66061465e-01  3.44406319e+02]
 [ 4.99976685e-01 -8.66038864e-01  6.12323400e-17  8.39997528e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
原始TCP位置: [915.249, 608.436, 233.411]
验证TCP位置: [915.280, 608.362, 233.397]
位置误差: 0.08 mm
姿态误差: 0.000239

完整关节角度:
关节1: 30.00°
关节2: 60.00°
关节3: 0.00°
关节4: 90.00°
关节5: 45.00°
关节6: 0.00°
